#Associating points to the grades in order to caluclate loan score

In [0]:
spark.conf.set("spark.sql.unacceptable_rated_pts",0)
spark.conf.set("spark.sql.very_bad_rated_pts",100)
spark.conf.set("spark.sql.bad_rated_pts",250)
spark.conf.set("spark.sql.good_rated_pts",500)
spark.conf.set("spark.sql.very_good_rated_pts",650)
spark.conf.set("spark.sql.excellent_rated_pts",800)

In [0]:
spark.conf.set("spark.sql.unacceptable_grade_pts",750)
spark.conf.set("spark.sql.very_bad_grade_pts",1000)
spark.conf.set("spark.sql.bad_grade_pts",1500)
spark.conf.set("spark.sql.good_grade_pts",2000)
spark.conf.set("spark.sql.very_good_grade_pts",2500)


In [0]:
bad_customer_data_final=spark.read\
    .format("csv")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("dbfs:/user/lendingclubproject/newbaddata/bad_data_customers_data_final")

In [0]:
bad_customer_data_final.createOrReplaceTempView("bad_data_customer")

In [0]:
#defining rating points
very_bad_rated_pts = 1
bad_rated_pts = 2
good_rated_pts = 3
very_good_rated_pts = 4
unacceptable_rated_pts = 0

# Step 2: Write your query with f-string substitution
ph_df = spark.sql(f"""
    SELECT c.member_id,
        CASE 
            WHEN p.last_payment_amount < (c.monthly_installment * 0.5) THEN {very_bad_rated_pts}
            WHEN p.last_payment_amount >= (c.monthly_installment * 0.5) AND p.last_payment_amount < c.monthly_installment THEN {bad_rated_pts}
            WHEN p.last_payment_amount = c.monthly_installment THEN {good_rated_pts}
            WHEN p.last_payment_amount > c.monthly_installment THEN {very_good_rated_pts}
            ELSE {unacceptable_rated_pts}
        END AS last_payment_pts
    FROM pranithdb_lending_club.loans_repayment AS p
    INNER JOIN pranithdb_lending_club.loans AS c ON c.loan_id = p.loan_id
    WHERE member_id NOT IN (SELECT member_id FROM bad_data_customer)
""")


In [0]:
ph_df.createOrReplaceTempView("ph_pts") 

In [0]:
%sql
select * from ph_pts limit 10

member_id,last_payment_pts
6d5091b3fcaaeb4eac37445042b6b79e8b21f16943b9c32c4f5dd74dcb0f2210,2
b5e7938b0a2da4ceaa75ea2e3c111d1e33ca0f944f954b6d65671ef395d21713,4
91060b858433e8a6107be9b76b324e304b5802631fc081b449188fe4b4e32205,4
cab1fa9f533688b0aab7c6be7b6256d15ad1dd5f3c4ca061a49cfeb26901977d,3
f74e401c1ab0adf788a7279dd70a27459fcc47733d35eed71149ff371d2a7283,4
8aef4bb29d609d8d684fd709af965bd690fc9ef3a2691777f38e0c10d622fe97,4
538b4653da3b1e8142b50baacf97a57b721eb6ec565f5fb63a1b71d191e9e934,4
b24d55f21390533c512f78f822153dc247e984dcd0cfe031c341ccef1c689817,4
1035c5401b0ca76d02c334b813302a9b72825e70b1659e8bc8ce5eea6c5d1f52,4
cb0f1777593e77909d0d359a6d9e633f28568500e17806f815395c92f0cd375d,4
